In [5]:
import numpy
import pandas as pd
import pandas.io.formats.style
from decimal import Decimal
from ipywidgets import HBox

def color_negative_red(value):
  if float(value) < 0:
    color = 'red'
  elif float(value) > 0:
    color = 'green'
  else:
    color = 'black'

  return 'color: %s' % color

def getResults(trades):
    rango = numpy.arange(current - (current / 4), current + (current / 4), 5)
    rango = list(rango)
    for trade in trades:
        if trade.has_key("base"):
            rango.append(trade["base"])
        if trade["type"] == "STOCK":
            rango.append(trade["price"])            
    rango = list(set(rango))
    rango.sort()
    tabla = []
    total_calls = 0
    total_puts = 0
    total_stocks = 0
    for i in rango:
        res = 0
        registro = {"indice": i, "valor": 0.0, "calls": 0, "puts": 0}
        for trade in trades:
            # acciones
            if trade["type"] == "STOCK":
                dif = (i - trade["price"])

                if(trade["price"] == i):
                    total_stocks = total_stocks + trade["qty"]
                res = res + (trade["qty"]*dif)                   
                registro["valor"] = round(res, 2)
            # opciones
            if trade.has_key("base"):
                dif = (i - trade["base"])
                if trade["type"] == "CALL": 
                    if(trade["base"] == i):
                        registro["calls"] = registro["calls"] + trade["qty"]
                        total_calls = total_calls + trade["qty"]
                    if trade["base"] > i:
                        res = res + (trade["price"]*trade["qty"]*100*-1)
                    else:
                        res = res + ((dif-trade["price"])*trade["qty"]*100)
                if trade["type"] == "PUT":
                    if(trade["base"] == i):
                        registro["puts"] = registro["puts"] + trade["qty"]
                        total_puts = total_puts + trade["qty"]
                    if trade["base"] > i:
                        res = res - ((dif + trade["price"])*trade["qty"]*100)
                    else:
                        res = res - (trade["price"]*trade["qty"]*100)
                registro["valor"] = round(res, 2)
        tabla.append([format(registro["indice"], '0.2f'), format(registro["valor"], '0.2f'), registro["calls"], registro["puts"]])
    
    print( "Acciones: {}".format(total_stocks))
    calls = "Calls: {}".format(total_calls)
    puts = "Puts: {}".format(total_puts)
    df = pd.DataFrame(tabla, columns=["Precio", "Valor", calls, puts])
    df = df.style.applymap(color_negative_red, subset=["Valor"])
    return df

def login_rodi(username, password):
    import requests
    import pickle
    import codecs
    from bs4 import BeautifulSoup

   # with open('cookies.jar', 'rb') as f:
   #     cookies = pickle.load(f)
   #     print(cookies)

    URL="https://rodi.sba.com.ar/Vistas/Login.aspx"
    URLGetEspecies="https://rodi.sba.com.ar/Vistas/PanelPersonal.aspx/GetEspecies"
    URLDataPack="https://rodi.sba.com.ar/Vistas/PanelPersonal.aspx/GetDataPack"
    headers={
        "User-Agent":"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.120 Safari/537.36",
        "Referer": "https://rodi.sba.com.ar/Vistas/PanelPersonal.aspx",
        "X-Request-Width": "XMLHttpRequest",
        "Host": "rodi.sba.com.ar"
    }    

    s=requests.Session()
    s.headers.update(headers)
    r=s.get(URL)
    soup=BeautifulSoup(r.content, "lxml")

    VIEWSTATE=soup.find(id="__VIEWSTATE")['value']
    VIEWSTATEGENERATOR=soup.find(id="__VIEWSTATEGENERATOR")['value']
    EVENTVALIDATION=soup.find(id="__EVENTVALIDATION")['value']

    login_data={"__LASTFOCUS":"",
    "__EVENTTARGET":"",
    "__EVENTARGUMENT":"",
    "__VIEWSTATE":VIEWSTATE,
    "__VIEWSTATEGENERATOR":VIEWSTATEGENERATOR,
    "__EVENTVALIDATION":EVENTVALIDATION,
    "ctl00$ContentPlaceHolder1$luLogin$txtUsuario": username,
    "ctl00$ContentPlaceHolder1$luLogin$txtClave":password,
    "ctl00$ContentPlaceHolder1$luLogin$ctl07":"Ingresar"}

    r=s.post(URL, data=login_data)

    # luego se llama al HTML de DataPack, sin esto
    # las sucesivas llamadas a DataPack no traen el JSON
    r=s.get(URLDataPack)

    with open('cookies.jar', 'wb') as f:
        pickle.dump(s.cookies, f)

    print("cookies guardadas en archivo cookies.jar")
    return
    
def datapack():
    import requests
    import pickle
    import json
    from bs4 import BeautifulSoup

    URLDataPack="https://rodi.sba.com.ar/Vistas/PanelPersonal.aspx/GetDataPack"
    headers={
        "User-Agent":"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.120 Safari/537.36",
        "Referer": "https://rodi.sba.com.ar/Vistas/PanelPersonal.aspx",
        "X-Request-Width": "XMLHttpRequest",
        "Host": "rodi.sba.com.ar"
    }

    s=requests.Session()

    with open('cookies.jar', 'rb') as f:
        s.cookies.update(pickle.load(f))

    # los tickers tienen que estar cargados previamente en RODI, de lo contrario esto no funciona.
    params={'obEstadoTabla' : {"TablaNombre":"tbEspecies","FiltroEspecies":"AF20_1#AF20_3#AO20_1#AO20_3#AY24_1#AY24_3#GFGC102.OC_0#GFGC105.OC_0#GFGC111.DI_0#GFGC111.OC_0#GFGC120.OC_0#GFGC132.DI_0#GFGC150.DI_0#GFGC63.0OC_0#GFGC75.0OC_0#GFGC80.0OC_0#GFGC87.0OC_0#GFGC90.0OC_0#GFGC93.0OC_0#GFGC99.0OC_0#GFGV102.DI_0#GFGV60.0OC_0#GFGV63.0OC_0#GFGV80.0OC_0#GFGV90.0DI_0#GFGV93.0OC_0#GFGV99.0OC_0#GGAL_3#GFGC102.DI_0","PagActualNro":"1","FilasxPagina":50}}
    # luego al JSON
    r=s.post(URLDataPack, json=params, headers=headers)

    with open('cookies.jar', 'wb') as f:
        pickle.dump(s.cookies, f)
    return json.loads(r.text)
    
def getRealtimeData(aData, aTicker, aVencimiento):
    for item in aData:
        if (item["Simbolo"] == aTicker) and (item["VencimientoID"] == aVencimiento):
            return item
